In [ ]:

%load_ext autoreload 
%autoreload 2
%matplotlib widget
%matplotlib inline
import cProfile #for checking the nr of calls and execution time
import pstats
from pstats import SortKey
import os
from tqdm.notebook import tqdm
import numpy as np
import multiple_planets_gas_acc as code_gas
from functions_pebble_accretion import *
from functions import *
import functions_plotting as plot
import matplotlib.pyplot as plt
import matplotlib as mpl
import astropy.units as u
import pandas as pd
from matplotlib.ticker import ScalarFormatter, LogFormatter, LogLocator, MultipleLocator, AutoMinorLocator
from matplotlib import cm, ticker
from matplotlib import colors
import matplotlib.gridspec as gridspec
import matplotlib.patches as patch
from matplotlib.offsetbox import AnchoredText
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import matplotlib.lines as mlines 
import sim_loader as sim_load


In [ ]:
# disc parameters
params_dict = {'St_const': None, 
                'M_dot_star': 0,
                'iceline_radius': None,
                'v_frag':(1*u.m/u.s).to(u.au/u.Myr).value,
                'alpha': 1e-2,
                'alpha_z': 1e-4, 
                'alpha_frag': 1e-4, 
                'H_r_model': 'Liu_mixed',
                'self_gravity': True,
                }

t_in = 1
t_fin = 5
# a_p0_in = np.geomspace(3, 0.1, num=5)
# # Combine the single value 20 with the generated array
# a_p0 = np.concatenate(([20], a_p0_in))
a_p0 = np.array([1, 0.5, 0.06])
#a_p0 = np.geomspace(60, 0.1, num = 5)
print(a_p0)
t0= ([t_in] * np.ones(len(a_p0))) # warning, this also goes in the initial conditions when doing mulitple planets otherwise it won't work

sim_params_dict = {'N_step': 10000,
                'a_p0':a_p0,
                't0':t0,
                't_in':t_in,
                't_fin': t_fin
                }

M_star = np.linspace(0.1, 1, num = 3)*const.M_sun.to(u.M_earth).value
print(M_star)
peb_acc = code_gas.PebbleAccretion(simplified_acc=False)
gas_acc = code_gas.GasAccretion()

for star_mass in M_star:
    params = code_gas.Params(**params_dict, star_mass=star_mass)
    print("star mass", params.star_mass*u.M_earth.to(u.M_sun))
    m0 = M0_pla_Mstar(a_p0, t0, sigma_gas_steady_state(a_p0, t0, params), params)
    print("m0", m0)
    sim_params = code_gas.SimulationParams(**sim_params_dict, m0 = m0)
    sim = code_gas.simulate_euler(migration=True, filtering=True, peb_acc=peb_acc, gas_acc=gas_acc, params=params, sim_params=sim_params, output_folder="sims/gas_acc/test/stellar_masses/gamma1_st1e-2")


In [ ]:
#folder_path = ["sims/gas_acc/test/stellar_masses", "sims/gas_acc/test/stellar_masses", "sims/gas_acc/test/stellar_masses"]
folder_path = ["sims/gas_acc/test/stellar_masses/gamma1_st1e-2", "sims/gas_acc/test/stellar_masses/gamma1_st1e-2", "sims/gas_acc/test/stellar_masses/gamma1_st1e-2"]

timestep = 10000
H_r_model = ['Liu_mixed', 'Liu_mixed', 'Liu_mixed']

mstar = [0.1,0.55,1.0]

simulations = []
sim_parameters = []
parameters = []

for folder_path, H_r_model, mstar in zip(folder_path, H_r_model, mstar):
    # List all files in the given folder
    all_files = os.listdir(folder_path)
    # create the list of names of the sim, sim_params and params files 
    sim_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('simulation_'+H_r_model) and f.endswith('N_steps'+str(timestep)+'_Mstar_'+str(mstar)+'.json')]
    sim_params_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('sim_params_'+H_r_model) and f.endswith('N_steps'+str(timestep)+'_Mstar_'+str(mstar)+'.json')]
    params_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('params_'+H_r_model) and f.endswith('N_steps'+str(timestep)+'_Mstar_'+str(mstar)+'.json')]

    # Sort the filenames based on the initial time
    sim_filenames.sort(key=sim_load.extract_initial_time)
    sim_params_filenames.sort(key=sim_load.extract_initial_time)
    params_filenames.sort(key=sim_load.extract_initial_time)


    # Load the simulations, sim_params, and params
    simulations.append([sim_load.JSONtoSimRes(filename) for filename in sim_filenames])
    sim_parameters.append([sim_load.load_sim_params(filename) for filename in sim_params_filenames])
    parameters.append([sim_load.load_params(filename) for filename in params_filenames])



fig, axs = plt.subplots(1,3, figsize=(19,6))
M_star = np.linspace(0.1, 1, num = 3)*const.M_sun.to(u.M_earth).value

print(len(simulations))

for i, (sim_list, param_list, sim_param_list) in enumerate(zip(simulations, parameters, sim_parameters)):
    print("i", i)
    for sim, params, sim_params in zip(sim_list, param_list, sim_param_list):
        plot.plot_growth_track_timescale(fig, axs[i], sim, params, sim_params,  migration=True, cmap=mpl.colormaps["inferno"].reversed(), add_ylabel=True if i ==0 else False,
                                        add_cbar=True if i ==0 else False, nofilter=False)
        axs[i].set_title(str(params.star_mass*u.M_earth.to(u.M_sun))+" $M_{\odot}$", fontsize = 25)
        pos = np.geomspace(4e-3, 1e2, num = 1000)
        M_iso_fin=M_peb_iso(pos, sim_params.t_fin, params)
        M_iso_in=M_peb_iso(pos, sim_params.t_in, params)
        axs[i].fill_between(pos, M_iso_fin, M_iso_in,  color='slateblue', alpha=0.1, zorder=10)
    plot.HD219134(axs[i], color = 'red')
    plot.solar_system(axs[i], color = 'green')
plt.savefig("figures/stellar_masses_gamma1_ste-2.png", dpi=300, bbox_inches='tight')

In [ ]:
folder_path = ["sims/gas_acc/test/stellar_masses/gamma1", "sims/gas_acc/test/stellar_masses/gamma1", "sims/gas_acc/test/stellar_masses/gamma1"]

timestep = 10000
H_r_model = ['irradiated', 'irradiated', 'irradiated']

mstar = [0.1,0.55,1.0]

simulations = []
sim_parameters = []
parameters = []

for folder_path, H_r_model, mstar in zip(folder_path, H_r_model, mstar):
    # List all files in the given folder
    all_files = os.listdir(folder_path)
    # create the list of names of the sim, sim_params and params files 
    sim_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('simulation_'+H_r_model) and f.endswith('N_steps'+str(timestep)+'_Mstar_'+str(mstar)+'.json')]
    sim_params_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('sim_params_'+H_r_model) and f.endswith('N_steps'+str(timestep)+'_Mstar_'+str(mstar)+'.json')]
    params_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('params_'+H_r_model) and f.endswith('N_steps'+str(timestep)+'_Mstar_'+str(mstar)+'.json')]

    # Sort the filenames based on the initial time
    sim_filenames.sort(key=sim_load.extract_initial_time)
    sim_params_filenames.sort(key=sim_load.extract_initial_time)
    params_filenames.sort(key=sim_load.extract_initial_time)


    # Load the simulations, sim_params, and params
    simulations.append([sim_load.JSONtoSimRes(filename) for filename in sim_filenames])
    sim_parameters.append([sim_load.load_sim_params(filename) for filename in sim_params_filenames])
    parameters.append([sim_load.load_params(filename) for filename in params_filenames])



fig, axs = plt.subplots(1,3, figsize=(19,6))
M_star = np.linspace(0.1, 1, num = 3)*const.M_sun.to(u.M_earth).value


for i, (sim_list, param_list, sim_param_list) in enumerate(zip(simulations, parameters, sim_parameters)):
    print("i", i)
    for sim, params, sim_params in zip(sim_list, param_list, sim_param_list):
        for p in range(sim_params.nr_planets-1,-1,-1):
            axs[i].loglog(sim.position[p], sim.mass[p])
        M_iso_fin=M_peb_iso(pos, sim_params.t_fin, params)
        axs[i].loglog(pos, M_iso_fin, color = 'slateblue')


In [ ]:
fig, axs = plt.subplots(1,2, figsize=(12,6))
r = 0.1
pos = np.geomspace(1e-2, 100, num =100)
time = 1
t = np.linspace(0.3, 10, num = 100)
Mstar = ([0.01, 0.1, 1]*u.M_sun).to(u.M_earth).value
ls = [':', '--', '-']
for i, star_mass in enumerate(Mstar):
    params = code_gas.Params(H_r_model='irradiated', star_mass = star_mass, M_dot_star=0, St_const=None, v_frag=(3*u.m/u.s).to(u.au/u.Myr).value)
    print(t)
    axs[0].loglog(t, st_frag_drift(r, t, params), linestyle = ls[i], label = str(star_mass*u.M_earth.to(u.M_sun))+" $M_{\odot}$", color = 'navy')
    axs[1].loglog(pos, st_frag_drift(pos, time, params), linestyle = ls[i], label = str(star_mass*u.M_earth.to(u.M_sun))+" $M_{\odot}$", color = 'gold')
for i  in range (2):
    axs[i].tick_params(axis='both', which='major', size = 10)
    axs[i].tick_params(axis='both', which='minor', size = 8)
    axs[i].legend()
    axs[i].set_title(r'Stokes number at '+str(r)+' AU', fontsize=25)

axs[0].set_xlabel(r'$t$ [Myr]', fontsize=25)
axs[0].set_ylabel(r'St', fontsize=25)

axs[1].set_xlabel(r'$a_{p,0}$ AU', fontsize=25)


In [ ]:
#t = (np.geomspace(1e5, 1e7, num = 100)*u.yr).to(u.s)
t = 0.1
pos = np.geomspace(0.007, 100, num = 10000)
M_dot = ([1e-7, 1e-8, 1e-9]*const.M_sun.cgs/u.yr).to(u.M_earth/u.Myr).value
print(M_dot)
label_mdot = ["$10^{-7}$", "$10^{-8}$", "$10^{-9}$"]
v_frag = ([1, 10]*u.m/u.s).to(u.au/u.Myr).value
sim_params = code_gas.SimulationParams()    

fig, axs = plt.subplots(1,3, figsize = (23,8))

alpha_frag = 1e-4
alpha = 1e-3
epsilon_heat = 1
epsilon_el = 1
color = [ 'navy',  'orange']
label = ["strongly viscous + irradiated", "viscous + irradiated", "irradiated"]
ls = [':','--','-']
par = ["params_lam_extreme", "params_lam", "params_irr"]
vfg = ["v_f_1", "v_f_10"]
for m, mdot_star in enumerate(M_dot):
    si_sublimation_points = {100: [], 1000: []}
    h2o_sublimation_points = {100: [], 1000: []}

    crossing_points = {"v_f_1": {"params_irr":
                       {
                           "frag_si": [],
                           "frag_stokes": [],
                           "stokes_epstein": [],
                           "frag_epstein": [],
                           "epstein_si": [],
                           "irr_ext": [],

                       },
                       "params_lam":
                       {
                           "frag_si": [],
                           "frag_stokes": [],
                           "stokes_epstein": [],
                           "frag_epstein": [],
                            "epstein_si": [],
                            "irr_ext": [],



                       },
                       "params_lam_extreme":
                       {
                           "frag_si": [],
                           "frag_stokes": [],
                           "stokes_epstein": [],
                           "frag_epstein": [],
                           "epstein_si": [],
                            "irr_ext": [],


                       }},
                       "v_f_10": {"params_irr":
                       {
                           "frag_si": [],
                           "frag_stokes": [],
                           "stokes_epstein": [],
                           "frag_epstein": [],
                           "epstein_si": [],
                            "irr_ext": [],


                       },
                       "params_lam":
                       {
                           "frag_si": [],
                           "frag_stokes": [],
                           "stokes_epstein": [],
                           "frag_epstein": [],
                            "epstein_si": [],
                           "irr_ext": [],


                       },
                       "params_lam_extreme":
                       {
                           "frag_si": [],
                           "frag_stokes": [],
                           "stokes_epstein": [],
                           "frag_epstein": [],
                           "epstein_si": [],

                       }}
                       }
    for i, vel_frag in enumerate(v_frag):
        params_lam = code_gas.Params(v_frag = vel_frag, H_r_model='Lambrechts_mixed', alpha_frag = alpha_frag, alpha = alpha, M_dot_star=mdot_star)
        params_irr = code_gas.Params(v_frag = vel_frag, H_r_model='irradiated', alpha_frag = alpha_frag, alpha = alpha, M_dot_star=mdot_star)
        params_lam_extreme = code_gas.Params(v_frag = vel_frag, H_r_model='Lambrechts_mixed', epsilon_el =epsilon_el,epsilon_heat =epsilon_heat, alpha_frag = alpha_frag, alpha = alpha, M_dot_star=mdot_star)
        for j, params in enumerate([params_lam_extreme, params_lam, params_irr]):
            c = color[i]
            st_f = st_frag(pos, t, params)
            st_d_epstein = st_drift_epstein(pos, t, params)
            st_d_stokes = st_drift_stokes(pos, t, params)
            st = st_frag_drift(pos, t, params) #combines the stokes and epstein regime and fragmentation regime

            ### compute the Epstein limit
            mean_free_path = mfp(pos, t, sigma_gas_steady_state(pos, t, params), params)
            st_from_dim = st_from_r_peb(9/4*mean_free_path, pos, t, params)
            #axs[m].loglog(pos.to(u.au), r_peb_from_st(st_from_dim, pos, t, params), color = 'grey', linestyle = ls[j], alpha = 0.4, zorder = 0)
            # place the dot where they meet the drift epstein
            st_ep_lim_mask = np.argmin(st_from_dim < st_d_epstein)
            
            if st_d_epstein[st_ep_lim_mask] < st_f[st_ep_lim_mask]:
                axs[m].scatter(pos[st_ep_lim_mask], st_d_epstein[st_ep_lim_mask], color = 'black', zorder = 10, 
                            label = "Epstein/Stokes regime transition" if i == 1 and j == 0 and m==0 else None)
            
            ## magnetic cavity
            r_mag_cav = r_magnetic_cavity(t, params)
            axs[m].axvline(r_mag_cav, color = 'grey', linestyle = '-.', alpha = 0.09, zorder = 0)
            #axs[m].axvline(r_magnetic_cavity_gen(t, params).to(u.au).value, color = 'grey', linestyle = '-.', alpha = 0.4, zorder = 0)
            ## Silicate sublimation line: T = 1200 K
            # T = 1200
            # si_subl_line = iceline(t, T, params)
            # #axs[m].axvline(si_subl_line.to(u.au).value, color = 'red', linestyle = ls[j], alpha = 0.4, zorder = 0)        
            # ## Si sublimation mask
            # si_subl_point = np.argmin(pos < si_subl_line)
            # axs[m].scatter(si_subl_line, st[si_subl_point], color = 'red', label = "Si evaporation line" if i == 1 and j ==0 and m==0 else None, zorder = 20)
            # si_sublimation_points[vel_frag].append((si_subl_line, st[si_subl_point]))

            T = 170
            h2o_subl_line = iceline(t, T, params)
            #axs[m].axvline(h2o_subl_line.to(u.au).value, color = 'blue', linestyle = ls[j], alpha = 0.4, zorder = 0)        

            ## H2O sublimation mask
            h2o_subl_point = np.argmin(pos < h2o_subl_line)
            axs[m].scatter(h2o_subl_line, st[h2o_subl_point], color = 'deepskyblue', label = "water evaporation line" if i == 1 and j == 0 and m==0 else None, zorder = 20)
            h2o_sublimation_points[vel_frag].append((h2o_subl_line, st[h2o_subl_point]))


            # first we plot all the lines
            line_f, = axs[m].loglog(pos, st_f, label = 
                    str(label[j]) if  i == 0 else None, color = c, linestyle = ls[j])
            line_d, = axs[m].loglog(pos, st_d_epstein, color = 'black', linestyle = ls[j])
            line_d_st, = axs[m].loglog(pos, st_d_stokes, color = 'black',
                                        linestyle = ls[j])
            
            #now we handle the line masks
            frag_epstein_mask = st_f < st_d_epstein # returns True where fragmentation stokes is smaller than Epstein drift Stokes
            frag_stokes_mask = st_f < st_d_stokes
            frag_drift_mask = st_f < st #returns True where fragmentation stokes is smaller than drift stokes
            stokes_epstein_mask = st_d_epstein < st_d_stokes #returns true where epstein is smaller than stokes
            # si_subl_mask = pos > si_subl_line #returns position outside Si line
            #cancels lines where St_frag < St_drift_epstein and St_drift_epstein < St_drift_stokes and cancels the stuff inside Si sublimation
            # line_f.set_data(pos[si_subl_mask&(frag_stokes_mask|frag_epstein_mask)], st_f[si_subl_mask&(frag_stokes_mask|frag_epstein_mask)])
            #cancels line where St_drift_stokes>St_frag and St_drift_stokes<St_drift_epstein
            #line_d_st.set_data(pos.to(u.au)[stokes_epstein_mask], st_d_stokes[stokes_epstein_mask])
            #cancels line where St_drift_epstein>St_drift_stokes
            line_d.set_data(pos[~stokes_epstein_mask&~frag_epstein_mask], st_d_epstein[~stokes_epstein_mask&~frag_epstein_mask])
            line_d_st.set_data(pos(u.au)[~frag_stokes_mask&stokes_epstein_mask], st_d_stokes[~frag_stokes_mask&stokes_epstein_mask])
            
            #fill in the dictionary of crossing points
            # frag_si_pt = np.argmin(st_f < si_subl_line)
            frag_stokes_pt = np.argmin(st_f < st_d_stokes)
            frag_epstein_pt = np.argmin(st_f < st_d_epstein)
            # epstein_si_pt = np.argmin(st_d_epstein < si_subl_line)
            crossing_points[str(vfg[i])][str(par[j])]["stokes_epstein"] = [pos[st_ep_lim_mask], st[st_ep_lim_mask]]
            # crossing_points[str(vfg[i])][str(par[j])]["frag_si"] = [pos[frag_si_pt], st[frag_si_pt]] 
            crossing_points[str(vfg[i])][str(par[j])]["frag_stokes"] = [pos[frag_stokes_pt], st[frag_stokes_pt]] 
            crossing_points[str(vfg[i])][str(par[j])]["frag_epstein"] = [pos[frag_epstein_pt], st[frag_epstein_pt]] 
            # crossing_points[str(vfg[i])][str(par[j])]["epstein_si"] = [pos[epstein_si_pt], st[epstein_si_pt]] 

        # the crossing points are required for the edges of the poligon to fill in
        irr_ext_pt = np.argmin(st_frag(pos, t, params_irr)< st_frag(pos, t, params_lam_extreme)) #to get the crossing points of visc heating with irradiation
        crossing_points[str(vfg[i])][str(par[j])]["irr_ext"] = [pos[irr_ext_pt], st_f[irr_ext_pt]] 
        ### filling the plots by fragmentation velocity
        if vel_frag == 1*u.m/u.s:
            st_f = st_frag(pos, t, params_lam_extreme)  
            st_f_irr = st_frag(pos, t, params_irr)      
            irr_mask = st_f < st_f_irr
            velocity_value = int(vel_frag)
            pts = np.array(si_sublimation_points[velocity_value])[::-1]
            xs = pos[irr_mask]
            above_red_line = np.where(xs<pts[:,0].max(), np.interp(xs, pts[:,0], pts[:,1]), 0)
            ymin = np.where(above_red_line!=0, above_red_line, st_f[irr_mask])
            axs[m].fill_between(xs, ymin, st_f_irr[irr_mask],  color='navy', alpha=0.1)

        else:
            st_f_d_ext = st_frag_drift(pos, t, params_lam_extreme)  
            st_d_irr = st_frag_drift(pos, t, params_irr)      
            epst_mask = st_f_d_ext < st_d_irr
            velocity_value = int(vel_frag)
            pts = np.array(si_sublimation_points[velocity_value])[::-1]
            xs = pos[epst_mask]
            above_red_line = np.where(xs<pts[:,0].max(), np.interp(xs, pts[:,0], pts[:,1]), 0)
            ymin = np.where(above_red_line!=0, above_red_line, st_f_d_ext[epst_mask])
            axs[m].fill_between(xs, ymin, st_d_irr[epst_mask], color='orange', alpha=0.1)

    # Plot the red segments connecting sublimation points and handle the fill_between logic
    for vel, points in si_sublimation_points.items():
        for i in range(len(points) - 1):
            x_values = [points[i][0], points[i + 1][0]]
            y_values = [points[i][1], points[i + 1][1]]
            axs[m].plot(x_values, y_values, color='red', linestyle='-', linewidth=2)
    """
    # Plot the blue segments connecting sublimation points and handle the fill_between logic
    for vel, points in h2o_sublimation_points.items():
        for i in range(len(points) - 1):
            x_values = [points[i][0], points[i + 1][0]]
            y_values = [points[i][1], points[i + 1][1]]
            axs[m].plot(x_values, y_values, color='blue', linestyle='-', linewidth=2)
    """

    axs[m].text(pos[1600], 1.6e-3, '$v_{\mathrm{frag}} = 1$ m/s', fontsize=20, color=color[0], rotation=28, rotation_mode='anchor', ha='left', va='bottom')
    axs[m].text(pos[400], 1.1e-1, '$v_{\mathrm{frag}} = 10$ m/s', fontsize=20, color=color[1], rotation=28, rotation_mode='anchor', ha='left', va='bottom')
    axs[m].text(pos[8000], 6e-2, 'drift limit', fontsize=20, color='black', rotation=-20, rotation_mode='anchor', ha='left', va='bottom')
    
    axs[m].text(r_mag_cav, 1.5e-4, '$r_{\mathrm{mag. cav.}}$', fontsize=20, color='grey', rotation=90, rotation_mode='anchor', ha='left', va='bottom', zorder = 0)

    axs[m].tick_params(axis = "both", which = "major", direction = 'in', size = 15)
    axs[m].tick_params(axis = "both", which = "minor", direction = 'in', size = 10)
    axs[m].tick_params(axis='y', labelsize= 17)
    axs[m].tick_params(axis='x', labelsize= 17)
    axs[m].set_xlabel("r [AU]", size = 25)
    axs[m].set_ylim(1e-4, 2)
    axs[m].set_xlim(3e-3, 3e2)
    #axs[m].set_title(r"$\dot{M}_*$ = "+label_mdot[m]+" $M_{\odot}$/yr", fontsize = 25)
    print(pos[0])
    axs[m].axvspan(3e-3, r_mag_cav, facecolor='none', hatch='/', edgecolor='gray', alpha =0.2)

for i, legd in enumerate(["$\dot{M}_* = 10^{-7} M_{\odot}$/yr", "$\dot{M}_* = 10^{-8} M_{\odot}$/yr", "$\dot{M}_* = 10^{-9} M_{\odot}$/yr"]):
    axs[i].annotate(legd, xy =(0.9, 0.9), xytext=(30,1), textcoords='offset points',
            size=24,
            bbox=dict(boxstyle="round", fc="lightgrey", alpha = 0.4,  ec="none"))


legend_main = axs[2].legend(loc= (0.40, 0.05),fontsize = 15)
axs[2].add_artist(legend_main)

"""Create the legend manually"""
legends = []
legends.append(plt.scatter(0, 0, marker = 'o',  color='red', s = 50, label='Si evaporation line'))
legends.append(plt.scatter(0,0, marker = 'o',  color='deepskyblue', s = 50, label='Water evaporation line'))
legends.append(plt.scatter(0,0,marker = 'o',  color='black', s = 50, label='Epstein/Stokes \ndrag regime transition'))
 
axs[1].legend(handles=legends, loc=(0.50, 0.05), frameon=True, prop={'size': 15}, ncol=1, labelcolor='black')
""""""

axs[0].set_ylabel("Stokes number", size = 25)
plt.tight_layout()
plt.subplots_adjust(wspace=0.12)
plt.savefig("figures/tests/St_frag_drift_mdot_1e3")

df = pd.DataFrame(si_sublimation_points)
df

In [ ]:
params_dict = {'St_const': None, 
                'M_dot_star': 0,
                #'M_dot_star': (1e-8*const.M_sun.cgs/u.yr).to(u.g/u.s),
                'iceline_radius': None,
                'alpha': 1e-2,
                'alpha_z': 1e-4, 
                'alpha_frag': 1e-4, 
                'iceline_alpha_change': False,
                'iceline_flux_change': False,
                'gas_accretion': False
                }
t_initial = 0.3
t_fin = 5
a_p0 = np.geomspace(50, 1e-1, num = 50)
t0= ([t_initial] * np.ones(len(a_p0))) # warning, this also goes in the initial conditions when doing mulitple planets otherwise it won't work
t_in = (t_initial) #is needed to start the simulation at the right time?
sim_params_dict = {'N_step': 10000,
                'a_p0':a_p0,
                't0':t0,
                't_in':t_in,
                't_fin': t_fin
                }

mstar = np.array([0.1, 0.55, 1.0])*const.M_sun.to(u.M_earth).value
output_folder = 'sims/gas_acc/accretion_maps/stellar_masses'

params_lam_ext = code_gas.Params(**params_dict, H_r_model='irradiated', star_mass=mstar[0])
params_lam = code_gas.Params(**params_dict, H_r_model='irradiated', star_mass=mstar[1])
params_irr = code_gas.Params(**params_dict, H_r_model='irradiated', star_mass=mstar[2])
peb_acc = code_gas.PebbleAccretion(simplified_acc=False)
gas_acc = code_gas.GasAccretion()

m0_lam_ext = M0_pla_Mstar(a_p0, t_in, sigma_gas_steady_state(a_p0, t_in, params_lam_ext), params_lam_ext)
sim_params_lam_ext = code_gas.SimulationParams(**sim_params_dict, m0=m0_lam_ext)
m0_lam = M0_pla_Mstar(a_p0, t_in, sigma_gas_steady_state(a_p0, t_in, params_lam), params_lam)
sim_params_lam = code_gas.SimulationParams(**sim_params_dict, m0=m0_lam)
m0_irr = M0_pla_Mstar(a_p0, t_in, sigma_gas_steady_state(a_p0, t_in, params_irr), params_irr)
sim_params_irr = code_gas.SimulationParams(**sim_params_dict, m0=m0_irr)

sim_lam_ext = code_gas.simulate_euler(migration=False, filtering=False, peb_acc=peb_acc,gas_acc = gas_acc,  params=params_lam_ext, sim_params=sim_params_lam_ext, output_folder=output_folder)
sim_lam = code_gas.simulate_euler(migration=False, filtering=False, peb_acc=peb_acc, gas_acc = gas_acc, params=params_lam, sim_params=sim_params_lam, output_folder=output_folder)
sim_irr = code_gas.simulate_euler(migration=False, filtering=False, peb_acc=peb_acc, gas_acc = gas_acc, params=params_irr, sim_params=sim_params_irr, output_folder=output_folder)


In [ ]:
sim_lam = sim_load.JSONtoSimRes("sims/gas_acc/accretion_maps/stellar_masses/simulation_irradiated_e_el_0.01_vfrag_1.0_planets_50_t0_0.3_N_steps10000_Mstar_0.1.json")
sim_lam_ext = sim_load.JSONtoSimRes("sims/gas_acc/accretion_maps/stellar_masses/simulation_irradiated_e_el_0.01_vfrag_1.0_planets_50_t0_0.3_N_steps10000_Mstar_0.55.json")
sim_irr = sim_load.JSONtoSimRes("sims/gas_acc/accretion_maps/stellar_masses/simulation_irradiated_e_el_0.01_vfrag_1.0_planets_50_t0_0.3_N_steps10000_Mstar_1.0.json")
sim_params_lam = sim_load.load_sim_params('sims/gas_acc/accretion_maps/stellar_masses/sim_params_irradiated_e_el_0.01_vfrag_1.0_planets_50_t0_0.3_N_steps10000_Mstar_0.1.json')
sim_params_lam_ext = sim_load.load_sim_params('sims/gas_acc/accretion_maps/stellar_masses/sim_params_irradiated_e_el_0.01_vfrag_1.0_planets_50_t0_0.3_N_steps10000_Mstar_0.55.json')
sim_params_irr = sim_load.load_sim_params('sims/gas_acc/accretion_maps/stellar_masses/sim_params_irradiated_e_el_0.01_vfrag_1.0_planets_50_t0_0.3_N_steps10000_Mstar_1.0.json')
params_lam = sim_load.load_params('sims/gas_acc/accretion_maps/stellar_masses/params_irradiated_e_el_0.01_vfrag_1.0_planets_50_t0_0.3_N_steps10000_Mstar_0.1.json')
params_lam_ext = sim_load.load_params('sims/gas_acc/accretion_maps/stellar_masses/params_irradiated_e_el_0.01_vfrag_1.0_planets_50_t0_0.3_N_steps10000_Mstar_0.55.json')
params_irr = sim_load.load_params('sims/gas_acc/accretion_maps/stellar_masses/params_irradiated_e_el_0.01_vfrag_1.0_planets_50_t0_0.3_N_steps10000_Mstar_1.0.json')


labels = [str(params_lam.star_mass*u.M_earth.to(u.M_sun))+' $M_{\odot}$', str(params_lam_ext.star_mass*u.M_earth.to(u.M_sun))+' $M_{\odot}$', str(params_irr.star_mass*u.M_earth.to(u.M_sun))+' $M_{\odot}$']
fig, axs = plt.subplots(1,3, figsize = (23,8))
num = 10000
clrs = ['grey','black']

# binning the mass on the y-axis between the min and max of the starting and final masses
m_min_lam = M0_pla(sim_lam.position[:,0].value, sim_params_lam.t_in, sigma_gas_steady_state(sim_lam.position[:,0].value, sim_params_lam.t_in, params_lam), params_lam).min()
m_min_lam_ext = M0_pla(sim_lam_ext.position[:,0].value, sim_params_lam_ext.t_in, sigma_gas_steady_state(sim_lam_ext.position[:,0].value, sim_params_lam_ext.t_in, params_lam_ext), params_lam_ext).min()
m_min_irr = M0_pla(sim_irr.position[:,0].value, sim_params_irr.t_in, sigma_gas_steady_state(sim_irr.position[:,0].value, sim_params_irr.t_in, params_irr), params_irr).min()

m_max_lam = M_peb_iso(sim_lam.position[:,0].value, sim_params_lam.t_in, params_lam).max()
m_max_lam_ext = M_peb_iso(sim_lam_ext.position[:,0].value, sim_params_lam_ext.t_in, params_lam_ext).max()
m_max_irr = M_peb_iso(sim_irr.position[:,0].value, sim_params_irr.t_in, params_irr).max()

M_in_min =  min(m_min_lam, m_min_lam_ext, m_min_irr)
M_fin_max = max(m_max_lam,m_max_lam_ext,m_max_irr) 
m_bins = np.geomspace(M_in_min, M_fin_max, num = num)
#print("mass binning", m_bins.to(u.M_earth))
Z = [np.zeros((len(m_bins), sim_params_lam.nr_planets))*u.Myr, np.zeros((len(m_bins), sim_params_lam.nr_planets))*u.Myr, np.zeros((len(m_bins), sim_params_lam.nr_planets))*u.Myr]
Zmin = np.array([])*u.Myr
Zmax = np.array([])*u.Myr
for i,(sim, params, sim_params) in enumerate(zip([sim_lam, sim_lam_ext, sim_irr],[params_lam, params_lam_ext, params_irr], [sim_params_lam, sim_params_lam_ext, sim_params_irr])):

    for p in range(sim_params.nr_planets): #loop over the nr_planets
        for m in range(len(m_bins)): #loops over the mass of the planet (timesteps)
            t_idx = np.argmax(sim.mass[p].value >= m_bins[m])
            Z[i][m,p] = sim.time[t_idx].to(u.Myr)
    axs[i].loglog(sim.position[:,0].to(u.au), M_peb_iso(sim.position[:,0].value, sim_params.t_in, params), 
            label = "M_peb_iso", color = 'slateblue', linestyle = ':') #caveat-> this is the M_iso at initial time, for the viscous case it decreases with time
    axs[i].loglog(sim.position[:,0], M_peb_iso(sim.position[:,0].value, sim_params.t_fin, params), 
            label = "M_peb_iso", color = 'slateblue', linestyle = ':') #caveat-> this is the M_iso at final time

    axs[i].loglog(sim.position[:,0], M0_pla_Mstar(sim.position[:,0].value, sim_params.t_in, 
                                                sigma_gas_steady_state(sim.position[:,0].value, sim_params.t_in, params), params), 
                                                label = "M_in", color = 'lightblue', linestyle = ':')
    #axs[i].loglog(sim.position[:,0].to(u.au), M_3D_2DH_trans(sim.position[:,0].to(u.au), sim_params.t_in, params).to(u.M_earth).value, color = 'lightgrey', linestyle = ':') #caveat, this is the transition mass for t_in, for viscous case it is time dependent
    #axs[i].loglog(sim.position[:,0].to(u.au), M_3D_2DH_trans(sim.position[:,0].to(u.au), sim_params.t_fin, params).to(u.M_earth).value, color = 'lightgrey', linestyle = ':') #caveat, this is the transition mass for t_in, for viscous case it is time dependent
    
    R_irr_visc_in = plot.r_visc_irr(sim.position[:,0].value, sim_params.t_in, params, sim_params)
    R_irr_visc_fin = plot.r_visc_irr(sim.position[:,-1].value, sim_params.t_fin, params, sim_params)
    for j,r in enumerate([R_irr_visc_in, R_irr_visc_fin]):
        col = clrs[j]
        if r is not None:
            print("transition radii"+str(params.H_r_model),r)
            axs[i].axvline(r, ymin = 0, ymax = 0.06, color = col,  alpha = 0.8, linewidth = 2)
            arrow_dx = r/2.5
            axs[i].annotate("", xy=(r-arrow_dx, 2e-6), xytext=(r, 2e-6), 
                         size="large",horizontalalignment="center", verticalalignment="center",
                         arrowprops = dict(arrowstyle = '->', color=col,alpha = 0.6, lw = 2))

    
    # Filling of the pebble isolation mass
    axs[i].fill_between(sim.position[:,0].to(u.au).value, M_peb_iso(sim.position[:,0].value, sim_params.t_in, params),
                         M_peb_iso(sim.position[:,0].value, sim_params.t_fin, params),  color='slateblue',alpha=0.3)
    Zmin = np.append(Zmin, Z[i].min())
    Zmax = np.append(Zmax, Z[i].max())

levs = np.geomspace(sim_params.t_in,sim_params.t_fin, num = 20)
vmin = sim_params_lam.t_in
vmax = sim_params_lam.t_fin
print(vmin, vmax)
for i,(sim, params, sim_params) in enumerate(zip([sim_lam, sim_lam_ext, sim_irr],[params_lam, params_lam_ext, params_irr], [sim_params_lam, sim_params_lam_ext, sim_params_irr])):

    X,Y = np.meshgrid(sim.position[:,0], m_bins)

    #plot white for the regions where the planet is dead or non existent
    Z[i][Z[i].value<=sim_params.t_in] = np.nan
    cmap = mpl.colormaps["inferno"].reversed()
    cmap.set_bad(color='white')
    masked_data = np.ma.masked_invalid(Z[i].value)
    print(masked_data)
    CS = axs[i].contourf(X, Y, masked_data, levels= levs, norm=colors.LogNorm(vmin = vmin, vmax = vmax), boundaries=levs, cmap = cmap)

#handling the colorbar	
#fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
cbar = fig.colorbar(CS, cax=cbar_ax)	
# Manually set the colorbar boundaries and ticks
cbar_ax.yaxis.set_major_locator(LogLocator(base=10.0, subs=[1.0,2.0, 5.0]))
cbar_ax.yaxis.set_major_formatter(LogFormatter())
cbar.ax.yaxis.set_major_formatter(plt.FuncFormatter(plot.custom_log_formatter))
cbar.set_label('accretion time [Myr]', fontsize=25, labelpad=15)
cbar.ax.tick_params(axis = 'both', which = 'major', size = 18, labelsize = 18)
cbar.ax.tick_params(axis = 'both', which = 'minor', size = 12)


for i in range(3):
    # Set the title and axis labels
    axs[i].set_title(labels[i], fontsize = 25)
    axs[i].set_ylim(M_in_min, M_fin_max)
    axs[i].set_xlabel('r [AU]', fontsize = 25, labelpad=20)
    axs[i].tick_params(axis='both', which='major', size=15)
    axs[i].tick_params(axis='both', which='minor', size=10)
    axs[i].tick_params(axis='both', which='major', labelsize=20)
    # axs[i].figure.axes[-1].yaxis.label.set_size(25)
    # axs[i].figure.axes[-1].tick_params(axis='both', which='minor', labelsize=10)
    # axs[i].figure.axes[-1].tick_params(axis='both', which='major', labelsize=15)
    axs[i].set_ylim(1e-6, 1e2)
    plot.all_y_ticks(axs[i], 100)


#panel 1
axs[0].text(sim.position[30,0].to(u.au).value, 1e-6, 'initial mass', fontsize=18, color='lightblue', rotation=36, rotation_mode='anchor', ha='left', va='bottom')
axs[0].text(sim.position[30,0].to(u.au).value, 3.5, 'pebble isolation mass', fontsize=18, color='slateblue', rotation=19, rotation_mode='anchor', ha='left', va='bottom')
#panel 2
axs[1].text(sim.position[30,0].to(u.au).value, 5e-6, 'initial mass', fontsize=18, color='lightblue', rotation=40.5, rotation_mode='anchor', ha='left', va='bottom')
axs[1].text(sim.position[30,0].to(u.au).value, 3.5, 'pebble isolation mass', fontsize=18, color='slateblue', rotation=19, rotation_mode='anchor', ha='left', va='bottom')
#panel 3
axs[2].text(sim.position[30,0].to(u.au).value, 3e-6, 'initial mass', fontsize=18, color='lightblue', rotation=40.5, rotation_mode='anchor', ha='left', va='bottom')
axs[2].text(sim.position[28,0].to(u.au).value, 3.5, 'pebble isolation mass', fontsize=18, color='slateblue', rotation=19, rotation_mode='anchor', ha='left', va='bottom')

axs[0].set_ylabel('M [$M_{\oplus}$]', fontsize = 25, labelpad=20)
plt.savefig("figures/heatmaps/stellar_masses", bbox_inches='tight')


In [ ]:
import json

#Add the Solar System planet images
#Uses the solution by Joe Kington from https://stackoverflow.com/questions/22566284/matplotlib-how-to-plot-images-instead-of-points
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

fig,ax = plt.subplots(figsize=(12.5,4.5*1.5*1.25))


def imscatter(x, y, image, ax=None, zoom=1):
    if ax is None:
        ax = plt.gca()
    try:
        image = plt.imread(image)
    except TypeError:
        # Likely already an array...
        pass
    im = OffsetImage(image, zoom=zoom)
    x, y = np.atleast_1d(x, y)
    artists = []
    for x0, y0 in zip(x, y):
        ab = AnnotationBbox(im, (x0, y0), xycoords='data', frameon=False)
        artists.append(ax.add_artist(ab))
    ax.update_datalim(np.column_stack([x, y]))
    ax.autoscale()
    return artists

planetsize=0.1
imscatter([0.387098],[0.055], 'roman_sensitivity/mercury.png',ax=ax,zoom=planetsize*0.3)
imscatter([0.723332],[0.815], 'roman_sensitivity/venus.png',ax=ax,zoom=planetsize*0.9)
imscatter([1],[1],'roman_sensitivity/Earth_Western_Hemisphere_transparent_background.png',ax=ax,zoom=planetsize)
imscatter([1.523679],[0.107], 'roman_sensitivity/mars.png',ax=ax,zoom=planetsize*240/500.0*0.9)
imscatter([5.204267],[317.8],'roman_sensitivity/jupiter.png',ax=ax,zoom=planetsize)
imscatter([9.582017],[95.152],'roman_sensitivity/saturn.png',ax=ax,zoom=planetsize*1.35)
imscatter([19.229411],[15.91],'roman_sensitivity/uranus.png',ax=ax,zoom=planetsize)
imscatter([30.103662],[17.147],'roman_sensitivity/neptune.png',ax=ax,zoom=planetsize)

imscatter([1],[0.7349/59.736],'roman_sensitivity/moon.png',ax=ax,zoom=planetsize*0.5)
imscatter([5.204267],[1.4819/59.736],'roman_sensitivity/ganymede.png',ax=ax,zoom=planetsize)
imscatter([9.582017],[1.346/59.736],'roman_sensitivity/titan.png',ax=ax,zoom=planetsize*0.22)


#Axis limits
Mmin=0.01
Mmax=10000 #15000
amin=0.009
amax=100

mjup=317.83
msun=1/3.0024584e-6   


amin=0.009
amax=100


## ROMAN SENSITIVITY LINE
nroaltparfile = 'roman_sensitivity/fitmaplimitsolns_NRO_layout_7f_3_covfac_52_3.json'
nroaltpars = json.load(open(nroaltparfile,'r'))

def nroalt(x,pars):
    return pars['a'] + pars['b']*x + pars['g']*np.sqrt(pars['d']**2+(x-pars['e'])**2)

fittedx = np.arange(np.log10(amin)-1,np.log10(amax)+1,0.05)
fittedline=nroalt(fittedx,nroaltpars)
ax.plot(10**fittedx,10**fittedline,'-',color='b',lw=3)


### KEPLER SENSITIVITY LINE
#The Kepler line
kepx = np.arange(np.log10(amin),np.log10(amax),0.05)
def kep(x):
    ret = np.zeros(x.shape) + Mmax*100.1
    xx = 10**x
    xm = (xx<1.2)
    ret[xm] = 0.68*xx[xm]**0.75
    return ret 

def kepburke2015(x):
    ret = np.zeros(x.shape) + Mmax*100.1
    xx = 10**x
    a0 = (530.0/365.25)**(2.0/3.0)
    xm = (xx<a0)
    ret[xm] = 2.2*(xx[xm]/a0)**0.75
    return ret 

ax.plot(10**kepx,kepburke2015(kepx),'-',color='r',lw=3)



smap = np.loadtxt('roman_sensitivity/all.magrid.NRO.layout_7f_3_covfac.52.filled') 
x = 10**smap[:33,0]
y = 10**smap[::33,1]
print(smap.shape,x.shape[0]*y.shape[0])
X,Y=np.meshgrid(x,y)
z = smap[:,2].reshape(X.shape) 
print("x",x)
print("y",y)
#Contours in log sensitivity
cf = ax.contourf(X,Y,z,cmap='Blues',levels=[-1,-0.5,0,0.5,1,1.5,2,2.5,3,3.5,4,4.5],vmin=-1,vmax=8)
cbar = plt.colorbar(cf,ax=ax,label='$Roman$ Sensitivity $-$ the number of planet detections\n expected if there is 1 planet per star at $(a,M_{\\rm p})$',ticks=[-1,0,1,2,3,4])
cbar.ax.set_yticklabels(['0.1','1','10','100','1000','10000'])


ax.set_axisbelow(False)
ax.set_xlabel('Semimajor Axis in AU')
ax.set_ylabel('Planet Mass in Earth Masses')
ax.set_xscale("log")
ax.set_yscale("log")
ax.set_xlim([amin,amax])
ax.set_ylim([Mmin,Mmax])
import matplotlib.ticker as ticker
ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))
ax.legend(loc=3,mode="expand",bbox_to_anchor=(0.0,0.995,1.0,0.102),ncol=3,fontsize=12,numpoints=1,handletextpad=-0.5)
plt.tight_layout()


plt.savefig("figures/roma_sensitivity", dpi=300)


In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(12,8))
plot.plot_roman_sensitivity(fig,axs)